As found in the [previous notebook](01_Data_Exploration.ipynb), the column "diagnosis" is missing from the pkl files, because tokenization didn't run. Let's see by running some experiments using the author's code what files and columns are missing in particular. So, we can narrow down the problem and fix it.

## Imports

In [2]:
import subprocess
from pathlib import Path

## Paths

## Experiments

### Pretraining CodeEmb with word2vec

In [ ]:
data = 
command = "python main.py \
    --distributed_world_size $WORLDSIZE \
    --input_path /path/to/data \
    --src_data $data \
    --task w2v \
    --model codeemb"
subprocess.run(python -m torch.distributed.launch --nproc_per_node=4 --use_env main.py \
    --distributed_world_size 4 \
    --input_path /path/to/data \
    --src_data $data\
    --task w2v\
    --model codeemb)